In [1]:
from datetime import datetime
import pandas as pd


In [2]:
#Create a dictionary that will specify the datatypes for the columns in our df

data_type ={
    
    '_CustomerID':str,
    }

In [3]:
#Read the data

data = pd.read_csv('sales_data.csv', dtype= data_type, parse_dates=['OrderDate'])

In [4]:
data.head()

,OrderNumber,Sales Channel,WarehouseCode,ProcuredDate,OrderDate,ShipDate,DeliveryDate,CurrencyCode,_SalesTeamID,_CustomerID,_StoreID,_ProductID,Order Quantity,Discount Applied,Unit Price,Unit Cost
0,SO - 000101,In-Store,WARE-UHY1004,12/31/2017,2018-05-31,6/14/2018,6/19/2018,USD,6,15,259,12,5,0.075,1963.1,1001.181
1,SO - 000102,Online,WARE-NMK1003,12/31/2017,2018-05-31,6/22/2018,7/2/2018,USD,14,20,196,27,3,0.075,3939.6,3348.660
2,SO - 000103,Distributor,WARE-UHY1004,12/31/2017,2018-05-31,6/21/2018,7/1/2018,USD,21,16,213,16,1,0.050,1775.5,781.220
3,SO - 000104,Wholesale,WARE-NMK1003,12/31/2017,2018-05-31,6/2/2018,6/7/2018,USD,28,48,107,23,8,0.075,2324.9,1464.687
4,SO - 000105,Distributor,WARE-NMK1003,4/10/2018,2018-05-31,6/16/2018,6/26/2018,USD,22,49,111,26,8,0.100,1822.4,1476.144


In [5]:
#Make a copy of the data object, so that you can do all the edits in the copy

df= data.copy()

In [6]:
print(df.shape)

(7991, 16)


In [7]:
#Our cols of interest:
# OrderNumber - to know the frequency of purchase by performing a count
# CustomerId - to group the customers
# OrderDate - Recency
# Orderquanity, DiscountApplied, UnitPrice, UnitCost - to calculate the monetary value


In [8]:
print(df.dtypes)

OrderNumber                 object
Sales Channel               object
WarehouseCode               object
ProcuredDate                object
OrderDate           datetime64[ns]
ShipDate                    object
DeliveryDate                object
CurrencyCode                object
_SalesTeamID                 int64
_CustomerID                 object
_StoreID                     int64
_ProductID                   int64
Order Quantity               int64
Discount Applied           float64
Unit Price                 float64
Unit Cost                  float64
dtype: object


In [9]:
df.head(10)

,OrderNumber,Sales Channel,WarehouseCode,ProcuredDate,OrderDate,ShipDate,DeliveryDate,CurrencyCode,_SalesTeamID,_CustomerID,_StoreID,_ProductID,Order Quantity,Discount Applied,Unit Price,Unit Cost
0,SO - 000101,In-Store,WARE-UHY1004,12/31/2017,2018-05-31,6/14/2018,6/19/2018,USD,6,15,259,12,5,0.075,1963.1,1001.181
1,SO - 000102,Online,WARE-NMK1003,12/31/2017,2018-05-31,6/22/2018,7/2/2018,USD,14,20,196,27,3,0.075,3939.6,3348.660
2,SO - 000103,Distributor,WARE-UHY1004,12/31/2017,2018-05-31,6/21/2018,7/1/2018,USD,21,16,213,16,1,0.050,1775.5,781.220
3,SO - 000104,Wholesale,WARE-NMK1003,12/31/2017,2018-05-31,6/2/2018,6/7/2018,USD,28,48,107,23,8,0.075,2324.9,1464.687
4,SO - 000105,Distributor,WARE-NMK1003,4/10/2018,2018-05-31,6/16/2018,6/26/2018,USD,22,49,111,26,8,0.100,1822.4,1476.144
5,SO - 000106,Online,WARE-PUJ1005,12/31/2017,2018-05-31,6/8/2018,6/13/2018,USD,12,21,285,1,5,0.050,1038.5,446.555
6,SO - 000107,In-Store,WARE-XYS1001,12/31/2017,2018-05-31,6/8/2018,6/14/2018,USD,10,14,6,5,4,0.150,1192.6,536.670
7,SO - 000108,In-Store,WARE-PUJ1005,4/10/2018,2018-05-31,6/26/2018,7/1/2018,USD,6,9,280,46,5,0.050,1815.7,1525.188
8,SO - 000109,In-Store,WARE-PUJ1005,12/31/2017,2018-06-01,6/16/2018,6/21/2018,USD,4,9,299,47,4,0.300,3879.3,2211.201
9,SO - 000110,In-Store,WARE-UHY1004,12/31/2017,2018-06-01,6/29/2018,7/1/2018,USD,10,33,261,13,8,0.050,1956.4,1212.968


In [20]:
#create a revenue column
#Unit price - discount - unit cost then multiply it by order quantity

df['Revenue']= (df['Unit Price'] - df['Unit Cost'] - df['Discount Applied']) * df['Order Quantity']


In [11]:
#df.dtypes

In [12]:
# Specify only the columns that we require for our analysis 
columns = ['OrderNumber', '_CustomerID', 'OrderDate', 'Revenue']
df_dataset = df[columns]

In [14]:
today_date = pd.to_datetime('2021-01-01')
rfm_dataset = df_dataset.groupby('_CustomerID').agg({
    'OrderDate': lambda v: (today_date - v.max()).days, 
    'OrderNumber': 'count',
    'Revenue': 'sum'
}
)

In [18]:
rfm_dataset.rename(
    columns = {
        'OrderDate': 'Recency',
        'OrderNumber': 'Frequency',
        'Revenue': 'MonetaryValue'
    },
    inplace= True
)

In [19]:
rfm_dataset.head(5)

,Recency,Frequency,MonetaryValue
_CustomerID,,,
1,9,152,483123.679
10,15,158,618229.422
11,6,178,712004.409
12,3,210,852066.815
13,4,171,636025.572


In [21]:
r = pd.qcut(rfm_dataset['Recency'], q=5, labels=range(5,0,-1))
f = pd.qcut(rfm_dataset['Frequency'], q=5, labels=range(1,6))
m = pd.qcut(rfm_dataset['MonetaryValue'], q=5, labels=range(1,6))

In [23]:
rfm = rfm_dataset.assign(R = r.values, F = f.values, M = m.values)


In [25]:
rfm['rfm_group '] = rfm[['R', 'F', 'M']].apply(lambda v: '-'.join(v.astype(str)), axis=1)
rfm['rfm_score_total'] = rfm[['R', 'F', 'M']].sum(axis=1)

In [26]:
rfm

,Recency,Frequency,MonetaryValue,R,F,M,rfm_group,rfm_score_total
_CustomerID,,,,,,,,
1,9,152,483123.679,2,2,1,2-2-1,5
10,15,158,618229.422,1,3,3,1-3-3,7
11,6,178,712004.409,3,5,5,3-5-5,13
12,3,210,852066.815,5,5,5,5-5-5,15
13,4,171,636025.572,4,4,4,4-4-4,12
14,5,157,571313.238,3,3,2,3-3-2,8
15,4,142,607970.850,4,1,3,4-1-3,8
16,3,135,576916.568,5,1,2,5-1-2,8
17,6,175,725296.821,3,5,5,3-5-5,13
